# Projekt «Booking.com»

## Die Idee

These: Die Hotel-Bewertungen auf Booking.com unterscheiden sich von Land zu Land. In Italien etwa sind die Bewertungen vieler Hotels in einem sehr hohen Bereich (ein Grossteil der Unterkünfte hat Note 8 aufwärts), während andere Länder sich im Bereich 6 bis 7 tummeln. Woran liegts? Woran krankts? Was gibt den Ausschlag für gute/schlechte Noten? Ist es die Infrastruktur, das Preis-Leistungsverhältnis, die Lage oder doch eher die Freundlichkeit des Personals? Hier ist die Schweiz ja offenbar nicht vorne dabei. Anhand der einzelnen Benotungskriterien lässt sich das herausfinden. Die Idee: Alle auf Booking.com gelisteten Hotels aller Länder auf die Bewertungen hin analysieren und dann evtl. mit anderen Parametern in Verbindung bringen, z.B. Lohnniveau. 

## 1. Vorbereitungen

### > alle nötigen Imports

Selenium, Scraping, BeautifulSoup, Pandas, Geopandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#für Selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### > Länderliste abspeichern, DataFrame erstellen aus Ländern und Links

Da ich später alle Länder separat aufrufen möchte, benötige ich eine Übersichtsliste aller Länder. Diese finde ich am einfachsten auf der Homepage http://www.booking.com und zwar ganz unten in der «Fussnote» unter «Länder» (https://www.booking.com/country.de.html?label=gen173rf-1FCAEoggI46AdIM1gDaCyIAQGYAQe4AQfIAQzYAQHoAQH4AQuIAgGiAg5sb2NhbGhvc3Q6ODg4OKgCAw;sid=bac122b1b039d543c04b493f5e62a162). Diese Länderseite rufe ich auf und speichere sie als HTML-File in den Projekt-Ordner unter «Booking.com_Countries.htm» auf meiner Festplatte ab. 

In [ ]:
# Übersichtsseite aller Länder einlesen

countries = open('Booking.com_Countries.htm', 'r')

# Seite lesbar machen mit BeautifulSoup

soup_countries = BeautifulSoup(countries, 'html.parser')

# Länder herausfiltern
countries = soup_countries.find_all('h2')

#Land herausfiltern
land = countries[1].text.replace('\n','')

#url zum Land herausfiltern
url = countries[1].find('a')['href'].split('?').pop(0)



#Dictionary erstellen

all_countries = []

for element in countries:
    try:
        country = element.text.replace('\n','')
        link = element.find('a')['href'].split('?').pop(0)
    except:
        None
        
    dict_countries = {'Land': country,
                     'Link': link}
    
    all_countries.append(dict_countries)

    

#DataFrame erstellen

df_all_countries = pd.DataFrame(all_countries)

# die letzten zwei Einträge sind keine Länder, brauche ich also nicht

df_all_countries = df_all_countries.head(195) 
df_all_countries


# 2. Selenium (...)

Nun steuere ich die einzelnen Links an, gebe das entsprechende Land in das Suchfeld ein und klicke «Zeig mir Preise» an, um auf die Übersicht aller Unterkünfte im betreffenden Land zu kommen. 

In [ ]:
#Selenium
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [ ]:
#Link zum Land ansteuern
driver.get(link)
#driver.get('https://www.booking.com/country/at.de.html')

#Suchfeld anwählen
suchfeld = driver.find_element_by_name("ss")

#Um zur Übersicht aller Hotels des Landes zu gelangen, 
#muss der Name des Landes ins Suchfeld eingegeben werden. 
#Ein Leerschlag führt jedoch ebenfalls zum Ziel. 
#bei beiden Varianten wird nur ein Teil der versprochenen Unterküfte ausgespuckt
#suchfeld.send_keys("country")
suchfeld.send_keys(" ")

#«Zeig mir Preise»-Button ansteuern, um zur Übersicht aller Unterkünfte zu gelangen
button = driver.find_element_by_class_name("sb-searchbox__button").click()

#Da immer nur 15 (oder 45 CHECK) Suchergebnisse pro Seite angezeigt werden: jeweils nächste-Seite-Button ansteuern
seite = driver.find_element_by_link_text("/searchresults.de.html?aid=304142&label=gen173nr-1DCAIoLDgcSAdYBGgsiAEBmAEHuAEHyAEM2AED6AEB-AECiAIBqAID&sid=e11e4e68c49d860e180be3ac386c1d0b&tmpl=searchresults&class_interval=1&dest_id=204&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=country&src_elem=sb&ss_all=0&ssb=empty&sshis=0&rows=15&offset=15").click()


# 3. Bewertungen pro Land im Detail